In [ ]:
import numpy as np
from scipy.optimize import root
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Brusselator model

The dynamics of the oscillating reaction discovered by Belousov and Zhabotinsky, can be modeled through the so-called Brusselator model depending on two parameters:

$$
\left\{\begin{aligned}
\mathrm{d}_t y_1 & = a - (b+1) y_1 + y_1^2y_2\\
\mathrm{d}_t y_2 & = b y_1 - y_1^2y_2
\end{aligned}\right.
$$

In [ ]:
class brusselator_model:

    def __init__(self, a, b) :
        self.a = a
        self.b = b
    
    def fcn(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b 
        y1_dot = a - (b+1)*y1 + y1*y1*y2 
        y2_dot = b*y1 - y1*y1*y2  
        return np.array([y1_dot, y2_dot])

    def jac(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b
        return np.array([[-(b+1)+2*y1*y2 , y1*y1], [b-2*y1*y2, -y1*y1]])

## Quasi-exact solution

The quasi-exact solution is obtained by using an explicit Runge-Kutta method of order 5 with stepsize control and fine tolerances due to Dormand and Prince.

In [ ]:
bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

tini = 0. 
tend = 40.

yini = (1.5, 3)

sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol_exa.t, y=sol_exa.y[0], mode="lines", name="y1"))
fig.add_trace(go.Scatter(x=sol_exa.t, y=sol_exa.y[1], mode="lines", name="y2"))
fig.update_layout(title="Quasi-exact Solution", xaxis_title="t", height=300, margin=dict(t=40, b=0))
fig.show()

fig_pp = go.Figure()
fig_pp.add_trace(go.Scatter(x=sol_exa.y[0], y=sol_exa.y[1]))
fig_pp.update_layout(title="Portrait phase", xaxis_title="u1", yaxis_title="u2", height=400, width=600, 
                     margin=dict(t=40, b=0))
fig_pp.show()

## Characterisation of stiffness

In [ ]:
jac = bm.jac

eig_vals = np.zeros((sol_exa.t.size, 2), dtype=np.complex_)
for it in range(sol_exa.t.size):
    eig_vals[it], _ = np.linalg.eig(jac(0, sol_exa.y[:,it]))

lambda1 = eig_vals[:, 0]
lambda2 = eig_vals[:, 1]

fig_eig_real = go.Figure()
fig_eig_real.add_trace(go.Scatter(x=sol_exa.t, y=np.real(lambda1), name="lamba1"))
fig_eig_real.add_trace(go.Scatter(x=sol_exa.t, y=np.real(lambda2), name="lamba2"))
fig_eig_real.update_layout(xaxis_title="t", height=300, margin=dict(t=40, b=0), title="Real part of eigen values")
fig_eig_real.show()

fig_eig_imag = go.Figure()
fig_eig_imag.add_trace(go.Scatter(x=sol_exa.t, y=np.imag(lambda1), name="lamba1"))
fig_eig_imag.add_trace(go.Scatter(x=sol_exa.t, y=np.imag(lambda2), name="lamba2"))
fig_eig_imag.update_layout(xaxis_title="t", height=300, margin=dict(t=40, b=0), title="Imaginary part of eigen values")
fig_eig_imag.show()

## Runge-Kutta methods

In [ ]:
class ode_result:
    def __init__(self, y, t, nfev):
        self.y = y
        self.t = t
        self.nfev = nfev 

def rk1(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*fcn(tn, yn)

    nfev = nt-1

    return ode_result(y, t, nfev)

def rk2(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        y[:,it+1] = yn + dt*k2

    nfev = 2*(nt-1)

    return ode_result(y, t, nfev)

def rk3(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        k3 = fcn(tn + dt, yn + dt*(-k1 + 2*k2))
        y[:,it+1] = yn + (dt/6)*(k1+4*k2+k3)

    nfev = 3*(nt-1)

    return ode_result(y, t, nfev)

def rk4(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        k3 = fcn(tn + 0.5*dt, yn + dt*(0.5*k2))
        k4 = fcn(tn + dt, yn + dt*k3)
        y[:,it+1] = yn + (dt/6)*(k1+2*k2+2*k3+k4)

    nfev = 4*(nt-1)

    return ode_result(y, t, nfev)

def rk(tini, tend, nt, yini, fcn, order):
    if order==1:
        return rk1(tini, tend, nt, yini, fcn)
    elif order==2:
        return rk2(tini, tend, nt, yini, fcn)
    elif order==3:
        return rk3(tini, tend, nt, yini, fcn)
    elif order==4:
        return rk4(tini, tend, nt, yini, fcn)
    else:
        print("Error : order not implemented")
        exit(-1) 

In [ ]:
nt = np.arange(1000, 10001, 1000)
#print(nt)

order = 1

fig = make_subplots(rows=2, cols=2, specs=[[{"colspan": 2}, None],[{}, {}]])

sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)
fig.add_trace(go.Scatter(x=sol_exa.t, y=sol_exa.y[0], line_color='grey', name='exact sol', legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol_exa.t, y=sol_exa.y[1], showlegend=False, line_color='grey', legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol_exa.y[0], y=sol_exa.y[1], showlegend=False, line_color='grey', legendgroup='1'), row=2, col=2)

sol_rk = rk(tini, tend, nt[0], yini, fcn, order)
sol_tmp = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk.t)
y1_err_rk = np.abs(sol_tmp.y[0] - sol_rk.y[0])
y2_err_rk = np.abs(sol_tmp.y[1] - sol_rk.y[1])
fig.add_trace(go.Scatter(x=sol_rk.t, y=sol_rk.y[0], name="y1", mode='markers', marker_symbol='x'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol_rk.t, y=sol_rk.y[1], name="y2", mode='markers', marker_symbol='x'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol_rk.t, y=y1_err_rk, name="err y1", mode='markers', marker_symbol='x'), row=2, col=1)
fig.add_trace(go.Scatter(x=sol_rk.t, y=y2_err_rk, name="err y2", mode='markers', marker_symbol='x'), row=2, col=1)
fig.add_trace(go.Scatter(x=sol_rk.y[0], y=sol_rk.y[1], mode='markers', marker_symbol='x', name="portrait phase"), row=2, col=2)


#create slider
steps = []
for nt_i in nt:
    sol_rk = rk(tini, tend, nt_i, yini, fcn, order)
    sol_tmp = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk.t)
    y1_err_rk = np.abs(sol_tmp.y[0] - sol_rk.y[0])
    y2_err_rk = np.abs(sol_tmp.y[1] - sol_rk.y[1])
    step = dict(method="update", label = f"{nt_i}", 
                args=[{"x": [sol_exa.t, sol_exa.t, sol_exa.y[0], sol_rk.t, sol_rk.t, sol_rk.t, sol_rk.t, sol_rk.y[0]], 
                       "y": [sol_exa.y[0], sol_exa.y[1], sol_exa.y[1], sol_rk.y[0], sol_rk.y[1], y1_err_rk, y2_err_rk, sol_rk.y[1]]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'nt = '}, steps=steps)]

fig.update_xaxes(range=[tini, tend], col=1, row=1, title='t')
fig.update_yaxes(col=1, row=1, title='yi')
fig.update_xaxes(range=[tini, tend], col=1, row=2, title='t')
fig.update_yaxes(col=1, row=2, title='|yi - yi exa|')
fig.update_xaxes(col=2, row=2, title='y1')
fig.update_yaxes(col=2, row=2, title='y2')
fig.update_layout(sliders=sliders, height=800, legend=dict(orientation="h",  y=1.05, xanchor="left", x=0))
fig['layout']['sliders'][0]['pad']=dict(t=50)
fig.show()